# REMoDNaV Error Analysis

In [1]:
import os
import copy
from typing import Literal, List

import numpy as np
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

import analysis._article_results.lund2013._helpers
import peyes

import analysis.utils as u
import analysis._article_results.lund2013._helpers as lund
import analysis._article_results.hfc._helpers as hfc
import peyes._utils.constants

pio.renderers.default = "browser"

C:\Users\jonathanni\PycharmProjects\pEYES\venv\Lib\site-packages\statsmodels\tools\tools.py:6: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 2.3.0)
  import scipy.linalg


### Set Constants

In [2]:
GT_ANNOTATORS = ["RA", "MN"]  # ground truth annotators
ONSET_STR, OFFSET_STR = peyes.constants.ONSET_STR, peyes.constants.OFFSET_STR

# visualization constants
GRID_LINE_COLOR, GRID_LINE_WIDTH = "lightgray", 1
ZERO_LINE_WIDTH = 2 * GRID_LINE_WIDTH

SINGLE_MEASURE_OPACITY, MEDIAN_OPACITY = 0.2, 1.0
SINGLE_MEASURE_LINE = dict(width=2 * GRID_LINE_WIDTH)
SINGLE_MEASURE_MARKER = dict(symbol="circle-open", size=6 * GRID_LINE_WIDTH,)
MEADIAN_LINE = dict(width=SINGLE_MEASURE_LINE['width'])
MEADIAN_MARKER = dict(symbol="circle", size=2 * SINGLE_MEASURE_MARKER['size'],)

FONT_FAMILY, FONT_COLOR = "Calibri", "black"
TITLE_FONT = dict(family=FONT_FAMILY, size=25, color=FONT_COLOR)
SUBTITLE_FONT = dict(family=FONT_FAMILY, size=20, color=FONT_COLOR)
AXIS_LABEL_FONT = dict(family=FONT_FAMILY, size=20, color=FONT_COLOR)
AXIS_TICK_FONT = dict(family=FONT_FAMILY, size=18, color=FONT_COLOR)
AXIS_LABEL_STANDOFF = 3

## Load Data

In [3]:
def load_labels(
        dataset_name: Literal["lund", "hfc"], labeler_subset: List[str] = None
) -> pd.DataFrame:
    if dataset_name == "lund":
        dataset_name = lund.DATASET_NAME
        directory = lund.PROCESSED_DATA_DIR
        stim_type = lund.STIMULUS_TYPE
        file_name = peyes.constants.LABELS_STR
    elif dataset_name == "hfc":
        dataset_name = hfc.DATASET_NAME
        directory = hfc.PROCESSED_DATA_DIR
        stim_type = hfc.STIMULUS_TYPE
        file_name = f"raw_{peyes.constants.LABELS_STR}"
    else:
        raise ValueError(f"Unknown dataset name: {dataset_name}")
    all = pd.read_pickle(os.path.join(directory, dataset_name, file_name + ".pkl"))
    iter1 = all.xs(1, level=peyes.constants.ITERATION_STR, axis=1)
    stim_trials = u.get_trials_for_stimulus_type(dataset_name, stim_type)
    stim_data = iter1.loc[:, stim_trials]

    labeler_subset = labeler_subset or ["RA", "MN", "engbert", "remodnav"]
    is_labeler_subset = np.isin(stim_data.columns.get_level_values(peyes.constants.LABELER_STR), labeler_subset)
    stim_data = stim_data.loc[:, is_labeler_subset]
    return stim_data

In [4]:
lund_labels = load_labels("lund",)
hfc_labels = load_labels("hfc")

## Label Error Analysis
#### (0) Visualization Helper Functions

In [5]:
def _add_distribution_bars(fig: go.Figure, df: pd.DataFrame, col_idx: int, showlegend: bool) -> go.Figure:
    df[peyes.constants.LABEL_STR] = df[peyes.constants.LABEL_STR].map(lambda lbl: peyes.parse_label(lbl))
    df[peyes.constants.LABELER_STR] = df[peyes.constants.LABELER_STR].map(
        lambda lblr: "REMoDNaV" if lblr == "remodnav" else f"Ann. {lblr}" if lblr in ["MN", "RA"] else lblr.upper()
    )
    for lbl in df[peyes.constants.LABEL_STR].unique():
        sub_data = df[df[peyes.constants.LABEL_STR] == lbl]
        label_name = peyes.parse_label(lbl).name
        fig.add_trace(
            go.Bar(
                x=sub_data[peyes.constants.LABELER_STR],
                y=sub_data["percentage"],
                name=label_name,
                legendgroup=label_name,
                offsetgroup=col_idx,
                marker_color=peyes._utils.visualization_utils._DEFAULT_COLORMAP[int(peyes.parse_label(lbl))],
                hovertemplate=f"{label_name}: %{{y:.2f}}%",
                showlegend=showlegend,
            ),
            row=1, col=col_idx + 1,
        )
    return fig


def _finalize_figure(fig: go.Figure, title: str, yaxis_label: str) -> go.Figure:
    fig.update_annotations(font=SUBTITLE_FONT, y=0.95, yref="paper", yanchor="bottom")
    fig.update_xaxes(
        title=dict(text=peyes.constants.LABELER_STR.title(), font=AXIS_LABEL_FONT, standoff=10),
        showline=False,
        showgrid=False, gridcolor=GRID_LINE_COLOR, gridwidth=GRID_LINE_WIDTH,
        zeroline=False, zerolinecolor=GRID_LINE_COLOR, zerolinewidth=ZERO_LINE_WIDTH,
        tickfont=AXIS_TICK_FONT,
    )
    fig.update_yaxes(
        title=dict(font=AXIS_LABEL_FONT, standoff=AXIS_LABEL_STANDOFF),
        showline=False,
        showgrid=True, gridcolor=GRID_LINE_COLOR, gridwidth=GRID_LINE_WIDTH,
        zeroline=True, zerolinecolor=GRID_LINE_COLOR, zerolinewidth=ZERO_LINE_WIDTH,
        tickfont=AXIS_TICK_FONT,
    )
    fig.update_yaxes(title_text=yaxis_label, row=1, col=1)
    fig.update_layout(
        barmode='stack',
        title=dict(text=title, font=TITLE_FONT),
        width=1000, height=400,
        plot_bgcolor='rgba(0, 0, 0, 0)',
        # paper_bgcolor='rgba(0, 0, 0, 0)',
        margin=dict(l=0, r=0, t=45, b=0, pad=0),
    )
    return fig

### (1) Label Distribution
We visualize the percentage of each label in the dataset, grouped by labeler.

In [6]:
def _calc_label_distribution(data: pd.DataFrame, labelers: list[str]) -> pd.DataFrame:
    def calc_single_labeler_dist(data: pd.DataFrame, labeler: str) -> pd.Series:
        percentages = (
            data
            .xs(labeler, level=peyes.constants.LABELER_STR, axis=1)
            .stack(future_stack=True)
            .value_counts()
            .sort_index()
        )
        percentages /= percentages.sum()
        percentages *= 100
        percentages.index = percentages.index.map(peyes.parse_label)
        return percentages

    df = pd.concat([calc_single_labeler_dist(data, lblr).rename(lblr) for lblr in labelers], axis=1).fillna(0)
    df = df.sort_index().stack(future_stack=True).reset_index()
    df.columns = [peyes.constants.LABEL_STR, peyes.constants.LABELER_STR, "percentage"]
    return df


column_titles = ["<b><i>lund2013</i></b>", "<b><i>HFC</i></b>"]
fig = make_subplots(
    rows=1, cols=len(column_titles), column_titles=column_titles,
    shared_xaxes=False, shared_yaxes=True,
    horizontal_spacing=0.025,
)

for c, ds in enumerate(column_titles):
    data = lund_labels if c == 0 else hfc_labels
    # Either calculate all labeler distributions...
    labelers = u.sort_labelers(data.columns.get_level_values(peyes.constants.LABELER_STR).unique())
    df = _calc_label_distribution(data, labelers)
    fig = _add_distribution_bars(fig, df, col_idx=c, showlegend=(c == 0))
fig = _finalize_figure(fig, title="Event Type Distribution", yaxis_label="Label Distribution (%)")

fig.show()

### (2) REMoDNaV SP Misclassification Analysis
Analyzing which GT labels were misclassified by REMoDNaV as _SPs_.

In [7]:
def _calc_sp_misclassifications(data: pd.DataFrame) -> pd.DataFrame:
    remodnav_labels = data.xs("remodnav", level=peyes.constants.LABELER_STR, axis=1)
    sp_indices = remodnav_labels[remodnav_labels == peyes.parse_label("smooth_pursuit")].stack().index
    data_on_sp = data.stack(peyes.constants.TRIAL_ID_STR, future_stack=True).loc[sp_indices]
    counts = pd.concat([
        data_on_sp[col].value_counts().rename(col) for col in GT_ANNOTATORS
    ], axis=1).fillna(0).sort_index()
    counts.columns = [f"Ann. {col}" for col in counts.columns]
    percentages = 100 * counts / counts.sum()
    df = percentages.stack(future_stack=True).reset_index()
    df.columns = [peyes.constants.LABEL_STR, peyes.constants.LABELER_STR, "percentage"]
    return df


column_titles = ["<b><i>lund2013</i></b>", "<b><i>HFC</i></b>"]
fig = make_subplots(
    rows=1, cols=len(column_titles), column_titles=column_titles,
    shared_xaxes=False, shared_yaxes=True,
    horizontal_spacing=0.025,
)

for c, ds in enumerate(column_titles):
    data = lund_labels if c == 0 else hfc_labels
    # Either calculate all labeler distributions...
    labelers = u.sort_labelers(data.columns.get_level_values(peyes.constants.LABELER_STR).unique())
    df = _calc_sp_misclassifications(data)
    fig = _add_distribution_bars(fig, df, col_idx=c, showlegend=(c == 0))
fig = _finalize_figure(fig, title="REModNaV SP Misclassifications", yaxis_label="Confused Label (%)")

fig.show()

## Fixation-Oriented REMoDNaV
We replace REMoDNaV's misclassified _SP_ labels with _fixation_ labels, and analyze the results with respect to the original GT labels.

In [8]:
import scipy.stats as stats

import analysis.process.sample_metrics as samp
import analysis.process.temporal_alignment as temp

from analysis._article_results.lund2013._helpers import LABELER_PLOTTING_CONFIG as lund_plotting_config
from analysis._article_results.hfc._helpers import LABELER_PLOTTING_CONFIG as hfc_plotting_config

LABELERS = GT_ANNOTATORS + ["engbert", "remodnav", "remodnav_new"]

In [9]:
def _replace_label(data: pd.DataFrame, labeler: str, old_label, new_label) -> pd.DataFrame:
    new_data = (
        data
        .copy(deep=True)
        .stack(peyes.constants.TRIAL_ID_STR, future_stack=True)
    )
    new_data[f"{labeler}_new"] = new_data[labeler].replace(peyes.parse_label(old_label), peyes.parse_label(new_label))
    new_data = new_data.unstack(peyes.constants.TRIAL_ID_STR)
    new_data.columns = new_data.columns.reorder_levels(data.columns.names)

    # add "iteration" level to the columns, with value 1 across all columns
    new_data[peyes.constants.ITERATION_STR] = 1
    new_data = new_data.set_index(peyes.constants.ITERATION_STR, append=True).unstack(peyes.constants.ITERATION_STR)
    return new_data

lund_labels2 = _replace_label(lund_labels, "remodnav", "smooth_pursuit", "fixation")
hfc_labels2 = _replace_label(hfc_labels, "remodnav", "smooth_pursuit", "fixation")
# replace non-fixation labels with 'undefined' label as HFC only includes fixation labels
hfc_labels2 = hfc_labels2.replace({
    lbl: peyes.parse_label("undef", safe=True) for lbl in np.unique(hfc_labels2.values)
    if lbl != peyes.parse_label("fixation", safe=False) or pd.isna(lbl)
})

### Sample-by-Sample Agreement

In [10]:
dataset_names = ["<b><i>lund2013</i></b>", "<b><i>HFC</i></b>"]
sample_metrics = {"cohen's_kappa": "<i>Cohen's Kappa</i>", "mcc": "<i>MCC</i>", "complement_nld": "<i>1-NLD</i>"}
figures = {
    gt: make_subplots(
        rows=len(sample_metrics), cols=len(dataset_names),
        row_titles=list(sample_metrics.values()), column_titles=dataset_names,
        shared_xaxes=True, shared_yaxes=True,
        horizontal_spacing=0.02, vertical_spacing=0.04,
    ) for gt in GT_ANNOTATORS
}

for c, ds in enumerate(dataset_names):
    if c == 0:
        dataset = lund_labels2
        plotting_config = lund_plotting_config
    else:
        dataset = hfc_labels2
        plotting_config = hfc_plotting_config
    data = samp.calculate_global_sample_metrics(dataset, GT_ANNOTATORS)
    for r, (metric_key, metric_name) in enumerate(sample_metrics.items()):
        for gt in GT_ANNOTATORS:
            curr_fig = figures[gt]
            for pred in LABELERS:
                if pred == gt:
                    continue  # skip the ground truth annotator
                if pred in GT_ANNOTATORS:
                    pred_name = f"Ann. {pred}"
                    pred_color = plotting_config["Other Human"][1]
                elif pred == "remodnav":
                    pred_name = "REMoDNaV"
                    pred_color = plotting_config[pred][1]
                elif pred == "remodnav_new":
                    pred_name = "REMoDNaV (No SPs)"
                    pred_color = "#DA9600"
                else:
                    pred_name = pred.upper()  # e.g., "engbert"
                    pred_color = plotting_config[pred][1]
                subset = (
                    data
                    .xs(1, level=peyes.constants.ITERATION_STR, axis=1)
                    .xs(gt, level=u.GT_STR, axis=1)
                    .xs(pred, level=u.PRED_STR, axis=1)
                    .xs(metric_key, axis=0)
                )
                if subset.empty:
                    continue
                curr_fig.add_trace(
                    row=r + 1, col=c + 1,
                    trace=go.Violin(
                        x=subset.values, y0=pred_name, name=pred, legendgroup=pred, scalegroup=metric_name,
                        line_color="black", fillcolor=pred_color,
                        box=dict(visible=False, width=0.95, line=dict(width=1)),
                        meanline=dict(visible=True, width=3, color=GRID_LINE_COLOR),
                        side="positive", width=1.8, points=False, opacity=1, spanmode="hard",
                        showlegend=(c == 0 and r == 0), visible=True,
                    )
                )

# update figure layouts
for gt, fig in figures.items():
    for ann in fig.layout.annotations:
        ann.textangle = 0
        ann.xref = ann.yref = "paper"
        ann.xanchor = "center"
        ann.yanchor = "bottom"
        if ann.text in dataset_names:
            ann.font = SUBTITLE_FONT
            ann.y = 1.01
            continue
        if ann.text in sample_metrics.values():
            ann.font = AXIS_LABEL_FONT
            idx = list(sample_metrics.values()).index(ann.text)
            ann.y = 0.635 if idx == 0 else 0.3 if idx == 1 else -0.1
            ann.x = 0.5
    fig.update_xaxes(
        showline=False,
        showgrid=False, gridcolor=GRID_LINE_COLOR, gridwidth=GRID_LINE_WIDTH,
        zeroline=False, zerolinecolor=GRID_LINE_COLOR, zerolinewidth=ZERO_LINE_WIDTH,
        tickfont=AXIS_TICK_FONT,
        range=[-0.02, 1.02]
    )
    fig.update_yaxes(
        title=dict(font=AXIS_LABEL_FONT, standoff=AXIS_LABEL_STANDOFF),
        showline=False,
        showgrid=True, gridcolor=GRID_LINE_COLOR, gridwidth=GRID_LINE_WIDTH,
        zeroline=False, zerolinecolor=GRID_LINE_COLOR, zerolinewidth=ZERO_LINE_WIDTH,
        tickfont=AXIS_TICK_FONT,
        range=[-0.1, len(LABELERS) + 0.02],
    )
    for r in range(len(sample_metrics)):
        fig.update_yaxes(title_text=peyes.constants.LABELER_STR.title(), row=r + 1, col=1)
    fig.update_layout(
        title=dict(
            text=f"Sample-by-Sample Agreement (GT: <i>{gt}</i>)",
            font=TITLE_FONT,
            xref="paper", xanchor="center", x=0.5,
            yref="container", yanchor="bottom", y=0.965,
            automargin=True,
        ),
        width=1200, height=600,
        plot_bgcolor='rgba(0, 0, 0, 0)',
        # paper_bgcolor='rgba(0, 0, 0, 0)',
        margin=dict(l=0, r=0, t=50, b=45, pad=0),
        legend=dict(
            font=AXIS_TICK_FONT,
            orientation="h",
            xref="paper", xanchor="center", x=0.5,
            yref="paper", yanchor="bottom", y=0.95,
        ),
    )

# show all figures
for gt, fig in figures.items():
    fig.show()

Sample Metrics :: Global: 100%|██████████| 10/10 [00:01<00:00,  9.22it/s]


#### Statistical Analysis

In [11]:
observations, statistics = dict(), dict()

for c, ds in enumerate(dataset_names):
    dataset = lund_labels2 if c == 0 else hfc_labels2
    ds_short = ds.replace("<b><i>", "").replace("</i></b>", "")
    data = samp.calculate_global_sample_metrics(dataset, GT_ANNOTATORS)
    for r, (metric_key, metric_name) in enumerate(sample_metrics.items()):
        for gt in GT_ANNOTATORS:
            subsets = {
                pred: (
                    data
                    .xs(1, level=peyes.constants.ITERATION_STR, axis=1)
                    .xs(gt, level=u.GT_STR, axis=1)
                    .xs(pred, level=u.PRED_STR, axis=1)
                    .xs(metric_key, axis=0)
                    .map(lambda x: np.round(x, 4))
                ) for pred in LABELERS if pred not in GT_ANNOTATORS
            }
            observations[(ds_short, gt, metric_key)] = subsets
            sub_stats = {
                "three_way": stats.friedmanchisquare(*subsets.values(), nan_policy="omit"),
                "engbert": stats.wilcoxon(np.round(subsets["remodnav_new"] - subsets["engbert"], 2), alternative="two-sided"),
                "remodnav": stats.wilcoxon(np.round(subsets["remodnav_new"] - subsets["remodnav"], 2), alternative="greater"),
            }
            statistics[(ds_short, gt, metric_key)] = pd.DataFrame({k: {"p": v[1], "statistic": v[0]} for k, v in sub_stats.items()})


# convert `statistics` to a DataFrame
statistics = pd.concat(statistics).stack(future_stack=True)
statistics.index.names = ["dataset", u.GT_STR, peyes.constants.METRIC_STR, "result_type", "test"]
statistics = statistics.unstack([peyes.constants.METRIC_STR, "result_type"])

statistics

Sample Metrics :: Global: 100%|██████████| 10/10 [00:01<00:00,  9.29it/s]


metric                cohen's_kappa                   mcc              \
result_type                       p   statistic         p   statistic   
dataset  gt test                                                        
HFC      MN three_way      0.000112   18.200000  0.000225   16.800000   
            engbert        0.014802    2.000000  0.027344    6.000000   
            remodnav       0.000977   55.000000  0.000977   55.000000   
         RA three_way      0.000225   16.800000  0.000225   16.800000   
            engbert        0.009766    3.000000  0.013672    4.000000   
            remodnav       0.000977   55.000000  0.000977   55.000000   
lund2013 MN three_way      0.000005   24.571429  0.000005   24.571429   
            engbert        0.005697    6.000000  0.003052    7.500000   
            remodnav       0.000061  105.000000  0.000061  105.000000   
         RA three_way      0.000003   25.600000  0.000002   26.759494   
            engbert        0.001150   11.000000  0.001659   17.000000   
            remodnav       0.000018  203.000000  0.000003  208.000000   

metric                complement_nld              
result_type                        p   statistic  
dataset  gt test                                  
HFC      MN three_way   5.004514e-04   15.200000  
            engbert     4.611350e-02    2.500000  
            remodnav    9.765625e-04   55.000000  
         RA three_way   3.707435e-04   15.800000  
            engbert     7.018226e-02    8.000000  
            remodnav    9.765625e-04   55.000000  
lund2013 MN three_way   8.781554e-06   23.285714  
            engbert     3.116095e-03    0.000000  
            remodnav    6.103516e-05  105.000000  
         RA three_way   8.741621e-07   27.900000  
            engbert     3.196572e-03    9.000000  
            remodnav    1.907349e-06  209.000000

### Boundary Sensitivity

In [12]:
METRIC = peyes.constants.D_PRIME_STR
dataset_names = ["<b><i>lund2013</i></b>", "<b><i>HFC</i></b>"]
boundary_types = [ONSET_STR, OFFSET_STR]
event_types = ["fixation", "saccade"]
figures = {
    (gt, evnt): make_subplots(
        rows=len(boundary_types), cols=len(dataset_names),
        row_titles=list(map(lambda bnd: f"{evnt} {bnd}".title(), boundary_types)), column_titles=dataset_names,
        shared_xaxes=True, shared_yaxes=True,
        horizontal_spacing=0.02, vertical_spacing=0.04,
    ) for gt in GT_ANNOTATORS for evnt in event_types
}

for evnt in event_types:
    for c, ds in enumerate(dataset_names):
        if c == 0:
            dataset = lund_labels2
            plotting_config = lund_plotting_config
        else:
            dataset = hfc_labels2
            plotting_config = hfc_plotting_config
        data = temp.signal_detection_metrics(
            dataset, np.arange(21), ["RA", "MN"], pos_labels=peyes.parse_label(evnt), dprime_correction="loglinear"
        )
        thresholds = data.index.get_level_values(peyes.constants.THRESHOLD_STR).unique().values
        for r, boundary in enumerate(boundary_types):
            for gt in GT_ANNOTATORS:
                curr_fig = figures[(gt, evnt)]
                for pred in LABELERS:
                    if pred == gt:
                        continue  # skip the ground truth annotator
                    if pred in GT_ANNOTATORS:
                        pred_name = f"Ann. {pred}"
                        pred_color = plotting_config["Other Human"][1]
                    elif pred == "remodnav":
                        pred_name = "REMoDNaV"
                        pred_color = plotting_config[pred][1]
                    elif pred == "remodnav_new":
                        pred_name = "REMoDNaV (No SPs)"
                        pred_color = "#DA9600"
                    else:
                        pred_name = pred.upper()  # e.g., "engbert"
                        pred_color = plotting_config[pred][1]
                    subset = (
                        data
                        .xs(1, level=peyes.constants.ITERATION_STR, axis=1)
                        .xs(METRIC, level=peyes.constants.METRIC_STR, axis=1)
                        .xs(gt, level=u.GT_STR, axis=1)
                        .xs(pred, level=u.PRED_STR, axis=1)
                        .xs(boundary, axis=0)
                    )
                    if subset.empty:
                        continue
                    curr_fig.add_trace(
                        row=r + 1, col=c + 1,
                        trace=go.Scatter(
                            x=thresholds, y=subset.mean(axis=1).values, error_y=dict(type="data", array=subset.sem(axis=1).values),
                            name=pred_name, legendgroup=pred_name,
                            mode="lines+markers",
                            marker=dict(size=5, color=pred_color),
                            line=dict(color=pred_color, dash="dot" if pred in GT_ANNOTATORS else None),
                            showlegend=(c == 0 and r == 0),
                        )
                    )

# update figure layouts
for (gt, evnt), fig in figures.items():
    for ann in fig.layout.annotations:
        ann.xref = ann.yref = "paper"
        ann.xanchor = "center"
        ann.yanchor = "bottom"
        if ann.text in dataset_names:
            ann.font = SUBTITLE_FONT
            ann.y = 0.975
            continue
        if evnt.title() in ann.text:
            ann.font = AXIS_LABEL_FONT
            ann.textangle = -90
            ann.x = 0.0
            ann.visible=False

    fig.update_xaxes(
        showline=False,
        showgrid=False, gridcolor=GRID_LINE_COLOR, gridwidth=GRID_LINE_WIDTH,
        zeroline=False, zerolinecolor=GRID_LINE_COLOR, zerolinewidth=ZERO_LINE_WIDTH,
    )
    for c in range(len(dataset_names)):
        fig.update_xaxes(
            title=dict(
                text="Δt (<i>ms</i>)",
                font=AXIS_LABEL_FONT, standoff=7.5
            ),
            tickfont=AXIS_TICK_FONT, tickmode="array", tickvals=np.arange(0, 21, 5),
            ticktext=[f"{(2*v):.1f}" for v in np.arange(0, 21, 5)] if c==0 else [f"{(3.33*v):.1f}" for v in np.arange(0, 21, 5)],
            row=len(dataset_names), col=c + 1
        )

    fig.update_yaxes(
        showline=False,
        showgrid=True, gridcolor=GRID_LINE_COLOR, gridwidth=GRID_LINE_WIDTH,
        zeroline=False, zerolinecolor=GRID_LINE_COLOR, zerolinewidth=ZERO_LINE_WIDTH,
        tickfont=AXIS_TICK_FONT,
    )
    for r in range(len(boundary_types)):
        fig.update_yaxes(
            title=dict(
                text=f"{ONSET_STR if r==0 else OFFSET_STR} Sensitivity (<i>d'</i>)",
                font=AXIS_LABEL_FONT, standoff=20,
            ),
            row=r + 1, col=1
        )

    fig.update_layout(
        title=dict(
            text=f"{evnt.title()} Boundary Sensitivity (GT: <i>{gt}</i>)",
            font=TITLE_FONT,
            xref="paper", xanchor="center", x=0.5,
            yref="container", yanchor="bottom", y=0.965,
            automargin=True,
        ),
        width=1200, height=600,
        plot_bgcolor='rgba(0, 0, 0, 0)',
        # paper_bgcolor='rgba(0, 0, 0, 0)',
        margin=dict(l=0, r=0, t=30, b=0, pad=0),
        legend=dict(
            font=AXIS_TICK_FONT,
            orientation="h",
            xref="paper", xanchor="center", x=0.5,
            yref="paper", yanchor="bottom", y=-0.175,
        ),
    )

# show all figures
for (gt, evnt), fig in figures.items():
    if evnt == "fixation":
        fig.show()

Samples Channel :: SDT Metrics: 100%|██████████| 10/10 [00:03<00:00,  3.05it/s]


#### Statistical Analysis

In [13]:
THRESHOLD = 5       # temporal threshold for analyzing d'
observations, statistics = dict(), dict()

for evnt in event_types:
    if evnt == "saccade":   # TODO: remove this to include saccade analysis
        continue
    for c, ds in enumerate(dataset_names):
        dataset = lund_labels2 if c == 0 else hfc_labels2
        ds_short = ds.replace("<b><i>", "").replace("</i></b>", "")
        data = temp.signal_detection_metrics(
            dataset, np.arange(21), ["RA", "MN"], pos_labels=peyes.parse_label(evnt), dprime_correction="loglinear"
        )
        for gt in GT_ANNOTATORS:
            for bndry in boundary_types:
                subsets = {
                    pred: (
                        data
                        .xs(1, level=peyes.constants.ITERATION_STR, axis=1)
                        .xs(METRIC, level=peyes.constants.METRIC_STR, axis=1)
                        .xs(gt, level=u.GT_STR, axis=1)
                        .xs(pred, level=u.PRED_STR, axis=1)
                        .xs(bndry, axis=0)
                        .loc[THRESHOLD]
                        .map(lambda x: np.round(x, 4))
                    ) for pred in LABELERS if pred not in GT_ANNOTATORS
                }
                observations[(ds_short, gt, evnt, bndry)] = subsets
                sub_stats = {
                    "three_way": stats.friedmanchisquare(*subsets.values(), nan_policy="omit"),
                    "engbert": stats.wilcoxon(np.round(subsets["remodnav_new"] - subsets["engbert"], 2), alternative="two-sided"),
                    "remodnav": stats.wilcoxon(np.round(subsets["remodnav_new"] - subsets["remodnav"], 2), alternative="greater"),
                }
                statistics[(ds_short, gt, evnt, bndry)] = pd.DataFrame({k: {"p": v[1], "statistic": v[0]} for k, v in sub_stats.items()})

# # convert `statistics` to a DataFrame
statistics = pd.concat(statistics).stack(future_stack=True)
statistics.index.names = ["dataset", u.GT_STR, peyes.constants.EVENT_STR, "boundary", "result_type", "test"]
statistics = statistics.unstack(["boundary", "result_type"])
statistics = statistics.droplevel(peyes.constants.EVENT_STR)    # TODO: remove this to include saccade analysis

statistics

Samples Channel :: SDT Metrics: 100%|██████████| 10/10 [00:06<00:00,  1.59it/s]


boundary                  onset                    offset          
result_type                   p   statistic             p statistic
dataset  gt test                                                   
HFC      MN three_way  0.000225   16.800000  2.248673e-04      16.8
            engbert    0.027344    5.500000  3.710938e-02       7.0
            remodnav   0.000977   55.000000  9.765625e-04      55.0
         RA three_way  0.000225   16.800000  3.707435e-04      15.8
            engbert    0.019531    5.000000  1.308594e-01      12.0
            remodnav   0.000977   55.000000  9.765625e-04      55.0
lund2013 MN three_way  0.000142   17.714286  7.485183e-05      19.0
            engbert    0.090576   25.000000  2.675781e-01      34.0
            remodnav   0.001160   98.000000  6.103516e-05     105.0
         RA three_way  0.000289   16.300000  7.157034e-07      28.3
            engbert    0.004221   31.000000  6.372261e-02      55.0
            remodnav   0.000052  199.000000  9.536743e-07     210.0